# Parse CSE File to Get Course List

In [ ]:
import PyPDF2
import pandas as pd

course_type = 'CSE'
pdfFileObj = open('ComputingScienceSummaries.pdf', 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
  
n_pages = len(pdfReader.pages)
text = ''

for n in range(n_pages):
    pageObj = pdfReader.pages[n]
    text += pageObj.extract_text()

pdfFileObj.close()

courses = {}
use_next_line = False

for i, line in enumerate(text.splitlines()):
    if line.strip().startswith(course_type) and line[9] == ' ':
        course = line.replace('  ', ' ').strip()
    if '©' in line and course_type in line:
        # 100 © Alberta Education, Alberta, Canada   Revised 2010  CSE2910:  CSE PROJECT B  
        course = course_type + line.split(course_type)[1]
        course = course.replace('  ', ' ').strip()
    if 'Prerequ' in line:
        preq = line.split(': ')[1].strip().replace(' ', '')
        if 'None' in preq:
            preq = ''
        courses[course] = [preq]
    if use_next_line:
        #print('True', line)
        preq2 = line.split(': ')[0].strip().replace(' ', '')
        if len(preq2) != 0: # in case there is no prereq on this line
            preq = preq + ',' + preq2
        courses[course] = [preq]
        use_next_line = False
    if 'Prerequisites' in line:
        #print('use next line', line)
        #print(i, course, preq, '---', line)
        use_next_line = True
    #if '1120' in line:
    #if '1010' in line:
    #    print(i, use_next_line, line)
cse = pd.DataFrame.from_dict(courses, orient='index', columns=['Prerequisites']).reset_index()
cse.columns = ['Course', 'Prerequisites']
cse

In [ ]:
for i, line in enumerate(text.splitlines()):
    if '©' in line:
        print(i, line)

In [ ]:
cse[cse['Prerequisites'].str.contains(course_type)]

In [ ]:
cse[cse['Course'].str[3]=='3']

In [ ]:
cse.to_csv(course_type+'.csv', index=False)

## Import from CSE.csv file

In [ ]:
import pandas as pd
cse = pd.read_csv('CSE.csv')
#df.sort_values('Course', ascending=False)

In [ ]:
# add graphviz to path
import os

# if it is Windows
if os.name == 'nt':
    os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

In [ ]:
import graphviz

dot = graphviz.Digraph(format='png') # or 'pdf' or 'svg'

for row in cse.itertuples():
    course_number = row.Course.split(':')[0]#[3:]
    if '950' not in course_number:
        if course_number[3] == '3':
            #dot.node(course_number, shape='diamond')
            shape = 'diamond'
        elif course_number[3] == '2':
            #dot.node(course_number, shape='ellipse')
            shape = 'ellipse'
        else:
            #dot.node(course_number, shape='box')
            shape = 'box'
        
        color = 'black'
        if course_number[4] == '9':
            color = 'red'

        dot.node(course_number, shape=shape, color=color)
        try:
            if len(row.Prerequisites) > 3:
                for preq in row.Prerequisites.split(','):
                    preq_number = preq#[3:]
                    dot.edge(preq_number, course_number)
        except:
            pass
dot

In [ ]:
# render the graph as a png
file_string = '../images/CSE'
dot.render(file_string)
import os
os.remove(file_string)

# NET Courses

In [ ]:
import PyPDF2
import pandas as pd

course_type = 'NET'
pdfFileObj = open('NetworkingSummary.pdf', 'rb')

pdfReader = PyPDF2.PdfReader(pdfFileObj)
n_pages = len(pdfReader.pages)
text = ''
for n in range(n_pages):
    pageObj = pdfReader.pages[n]
    text += pageObj.extract_text()
pdfFileObj.close()

courses = {}
use_next_line = False

for i, line in enumerate(text.splitlines()):
    if line.strip().startswith(course_type):# and line[9] == ' ':
        course = line.replace('  ', ' ').strip()
        use_next_line = False
    if '©' in line and course_type in line and ':' in line:
        course = line.split(':')[0].strip().split(' ')[-1] + ':' + line.split(':')[1].strip()
        use_next_line = False
    if 'Prerequ' in line:
        try:
            preq = line.split(':')[1].strip().replace(' ', '')
        except:
            preq = ''
        if 'None' in preq:
            preq = ''
        courses[course] = [preq]
    if use_next_line:
        preq2 = line.split(': ')[0].strip().replace(' ', '')
        if len(preq2) != 0: # in case there is no prereq on this line
            preq = preq + ',' + preq2
        courses[course] = [preq]
    if 'Prerequisites' in line:
        use_next_line = True
net = pd.DataFrame.from_dict(courses, orient='index', columns=['Prerequisites']).reset_index()
net.columns = ['Course', 'Prerequisites']
net

In [ ]:
net['Prerequisites'] = net['Prerequisites'].str.replace('NET2030','NET2030,NET2040,NET2050,NET2060,NET2070')

In [ ]:
net[net['Prerequisites'].str.contains(course_type)]

In [ ]:
net.to_csv(course_type+'.csv', index=False)

## Read from NET.csv

In [ ]:
net = pd.read_csv('NET.csv')

In [ ]:
import graphviz
dot = graphviz.Digraph(format='png') # or 'pdf' or 'svg'

for row in net.itertuples():
    course_number = row.Course.split(':')[0]
    if '950' not in course_number:
        if course_number[3] == '3':
            shape = 'diamond'
        elif course_number[3] == '2':
            shape = 'ellipse'
        else:
            shape = 'box'

        color = 'black'
        if course_number[4] == '9':
            color = 'red'

        dot.node(course_number, shape=shape, color=color)#, fontcolor='blue')
        try:
            if row.Prerequisites != '':
                for preq in row.Prerequisites.split(','):
                    dot.edge(preq, course_number)
        except:
            pass
dot

In [ ]:
# render the graph as a png
file_string = '../images/NET'
dot.render(file_string)
import os
os.remove(file_string)

# ELT Courses

In [ ]:
import PyPDF2
import pandas as pd

course_type = 'ELT'
pdfFileObj = open('ElectroTechnologiesSummaries.pdf', 'rb')

pdfReader = PyPDF2.PdfReader(pdfFileObj)
n_pages = len(pdfReader.pages)
text = ''
for n in range(n_pages):
    pageObj = pdfReader.pages[n]
    text += pageObj.extract_text()
pdfFileObj.close()

courses = {}
use_next_line = False

for i, line in enumerate(text.splitlines()):
    if line.strip().startswith(course_type):# and line[9] == ' ':
        course = line.replace('  ', ' ').strip()
        use_next_line = False
    if '©' in line and course_type in line and ':' in line:
        course = line.split(':')[0].strip().split(' ')[-1] + ':' + line.split(':')[1].strip()
        use_next_line = False
    if 'Prerequ' in line:
        try:
            preq = line.split(':')[1].strip().replace(' ', '')
        except:
            preq = ''
        if 'None' in preq:
            preq = ''
        courses[course] = [preq]
    if use_next_line:
        preq2 = line.split(': ')[0].strip().replace(' ', '')
        if len(preq2) != 0: # in case there is no prereq on this line
            preq = preq + ',' + preq2
        courses[course] = [preq]
    if 'Prerequisites' in line:
        use_next_line = True
elt = pd.DataFrame.from_dict(courses, orient='index', columns=['Prerequisites']).reset_index()
elt.columns = ['Course', 'Prerequisites']
elt

In [ ]:
elt.to_csv(course_type+'.csv', index=False)

## Read from ELT.csv

In [ ]:
import pandas as pd
elt = pd.read_csv('ELT.csv')

In [ ]:
recommended = ['1010','1130','1140','2010','2140','2160','2240','2170','3170','3150','3180','3190','3200','1910','2910','2920','3910','3920']
design_courses = ['DES1020','DES1040','DES2045']

import graphviz
dot = graphviz.Digraph(format='png') # or 'pdf' or 'svg'

for course_number in design_courses:
    if course_number[3] == '3':
        shape = 'diamond'
    elif course_number[3] == '2':
        shape = 'ellipse'
    else:
        shape = 'box'
    color = 'black'
    fontcolor = 'grey'
    dot.node(course_number, shape=shape, color=color, fontcolor=fontcolor)
    try:
        dot.edge(previous_course, course_number)
    except:
        pass
    previous_course = course_number

for row in elt.itertuples():
    course_number = row.Course.split(':')[0]
    if '950' not in course_number:
        if course_number[3:] in recommended:
            if course_number[3] == '3':
                shape = 'diamond'
            elif course_number[3] == '2':
                shape = 'ellipse'
            else:
                shape = 'box'

            fontcolor = 'black'

            color = 'black'
            if course_number[4] == '9':
                color = 'red'
            dot.node(course_number, shape=shape, color=color, fontcolor=fontcolor)

        try:
            if row.Prerequisites != '':
                for preq in row.Prerequisites.split(','):
                    if preq[3:] in recommended:
                        dot.edge(preq, course_number)
        except:
            pass
dot

In [ ]:
# render the graph as a png
file_string = '../images/ELT'
dot.render(file_string)
import os
os.remove(file_string)

# Combining All Together

In [ ]:
import pandas as pd

c = pd.read_csv('CSE.csv').fillna('')
n = pd.read_csv('NET.csv').fillna('')
e = pd.read_csv('ELT.csv').fillna('')

# removed unused ELT courses
recommended = ['1010','1130','1140','1910','2010','2140','2160','2240','2170','2910','2920','3170','3150','3180','3190','3200','3910','3920']
elt_recommended_list = ['ELT' + x for x in recommended]
e = e[e['Course'].str[:7].isin(elt_recommended_list)]

design_courses = {'DES1020: THE DESIGN PROCESS':'','DES1040: 3-D DESIGN 1':'DES1020','DES2045: 3-D DESIGN 2':'DES1040'}
d = pd.DataFrame.from_dict(design_courses, orient='index', columns=['Prerequisites']).reset_index()
d.columns = ['Course', 'Prerequisites']

df = pd.concat([c,e,n,d])
df.reset_index(drop=True, inplace=True)

# split column to course code and course name
df['Course Number'] = df['Course'].str.split(':').str[0].str.strip()
df['Course Name'] = df['Course'].str.split(':').str[1].str.strip()
df

In [ ]:
import graphviz

def coursesVisualized(filtered_df):
    dot = graphviz.Digraph(format='png') # or 'pdf' or 'svg'
    for row in filtered_df.itertuples():
        course_number = row.Course.split(':')[0]

        if course_number[3] == '3':
            shape = 'diamond'
        elif course_number[3] == '2':
            shape = 'ellipse'
        else:
            shape = 'box'

        fontcolor = 'black'
        color = 'black'
        #if course_number[4] == '9': # project courses
        #    color = 'red'
        dot.node(course_number, shape=shape, color=color, fontcolor=fontcolor)

        try:
            if row.Prerequisites != '':
                for preq in row.Prerequisites.split(','):
                    dot.edge(preq, course_number)
        except:
            pass
    display(dot)


coursesVisualized(df[df['Course Number'].str.contains('CSE')])

coursesVisualized(df[df['Course Number'].str.contains('DES') | df['Course Number'].str.contains('ELT')].sort_values(by='Course Number'))

coursesVisualized(df[df['Course Number'].str.contains('NET')])

In [ ]:
'''
#completed = ['CSE1010']
completed = []

import graphviz
dot = graphviz.Graph()

for row in elt.itertuples():
    course_number = row.Course
    if '950' not in course_number:
        if course_number in completed:
            color = 'red'
        else:
            color = 'black'
        if course_number[3] == '3':
            d = dot.node(course_number, shape='diamond',color=color)
            #d.attr('node', color='blue')
        else:
            d = dot.node(course_number, shape='box',color=color)
        
        if row.Prerequisites != '':
            for preq in row.Prerequisites.split(','):
                dot.edge(preq, course_number)

#dot.attr('graph', ratio='compress')
#dot.attr('graph', ratio='0.5')
#dot.attr('graph', size='80,80')
#dot.attr('graph', ratio='1.5')
#dot.attr('graph', layout='neato', defaultdist='2.0', overlap='false', splines='true', sep='+2.0', nodesep='0.5', ranksep='0.5', concentrate='true', size='8,8', ratio='0.5', orientation='landscape')
dot.attr('graph', layout='neato', pack='false', defaultdist='800')
#dot.attr('graph', orientation='landscape')
dot
'''

In [ ]:
# search through all .md files in the current directory for the string 'goals' and print the filename and line number
'''
import os
for root, dirs, files in os.walk(".."):
    for filename in files:
        if filename.endswith(".md"):
            with open(os.path.join(root, filename), encoding='latin1') as f:
                for i, line in enumerate(f):
                    if 'goals' in line:
                        print(os.path.join(root, filename), i, line)
'''